### 1. Import libraries

In [2]:
import torch 
import torch.nn as nn
import torchtext    
import os
import numpy as np
import pandas as pd
import spacy
import timm
import matplotlib.pyplot as plt

from PIL import Image
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchvision import transforms

/home/blackcait/.miniconda3/envs/pytorch_env/lib/python3.12/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/blackcait/.miniconda3/envs/pytorch_env/lib/python3.12/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/blackcait/.miniconda3/envs/pytorch_env/lib/python3.12/site-packages/torchte

### 2. load dataset

sample need 3 things: image, question, answer

In [3]:
val_data  = []
val_data_path = "vqa_coco_dataset/vaq2.0.DevImages.txt"
with open(val_data_path, "r") as f:
    lines = f.readlines()
    # print(lines[:1])
    for line in lines:
        temp = line.split('\t')
        qa = temp[1].split('?')
        if len(qa) == 3:
            answer = qa[2].strip() #strip() using for delete space
        else:
            answer = qa[1].strip()
        sample = {
            "image": temp[0][:-2],
            "question": qa[0] + "?",
            "answer": answer
        }
        val_data.append(sample)
print("validation: ",len(val_data))

test_data  = []
test_data_path = "vqa_coco_dataset/vaq2.0.TestImages.txt"
with open(test_data_path, "r") as f:
    lines = f.readlines()
    for line in lines:
        temp = line.split('\t')
        qa = temp[1].split('?')
        if len(qa) == 3:
            answer = qa[2].strip() #strip() using for delete space
        else:
            answer = qa[1].strip()
        sample = {
            "image": temp[0][:-2],
            "question": qa[0] + "?",
            "answer": answer
        }
        test_data.append(sample)
print("test",len(test_data))


train_data  = []
train_data_path = "vqa_coco_dataset/vaq2.0.TrainImages.txt"
with open(train_data_path, "r") as f:
    lines = f.readlines()
    for line in lines:
        temp = line.split('\t')
        qa = temp[1].split('?')
        if len(qa) == 3:
            answer = qa[2].strip() #strip() using for delete space
        else:
            answer = qa[1].strip()
        sample = {
            "image": temp[0][:-2],
            "question": qa[0] + "?",
            "answer": answer
        }
        train_data.append(sample)
print("train",len(train_data))

validation:  1952
test 2022
train 7846


In [26]:
answer = set([sample['answer'] for sample in train_data])
answer

{'no', 'yes'}

### 3. Data preprocessing

In [32]:
eng = spacy.load('en_core_web_sm')
# text = 'hello world'
# print([token for token in eng.tokenizer(text)])
def get_tokens(data_iters):
    for sample in data_iters:
        question = sample['question']
        yield [token.text for token in eng.tokenizer(question)]

vocab = build_vocab_from_iterator(
    get_tokens(train_data),
    min_freq= 1,
    specials = ['<unk>', '<sos>','<eos>','<pad>'],
    special_first=True
)
vocab.set_default_index(vocab['<unk>'])

In [34]:
def tokenize(question, max_seq_len = 20):
    tokens = [token.text for token in eng.tokenizer(question)]
    sequence = [vocab[token] for token in tokens]
    if len(sequence) < max_seq_len:
        sequence += [vocab['<pad>']] * (max_seq_len - len(sequence))
    else:
        sequence = sequence[:max_seq_len]
    return sequence
tokenize('hello_world')

[0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [8]:
classes = list([sample['answer'] for sample in train_data])
classes_to_idx = {
    class_name: idx for idx, class_name in enumerate(classes)
}
idx_to_class= {
    idx: class_name for idx, class_name in enumerate(classes)
}

In [12]:
print(classes)
print(classes_to_idx)


['no', 'no', 'yes', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'no', 'yes', 'no', 'no', 'no', 'yes', 'no', 'yes', 'no', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'yes', 'no', '

In [11]:
print(idx_to_class)


{0: 'no', 1: 'no', 2: 'yes', 3: 'no', 4: 'no', 5: 'no', 6: 'yes', 7: 'no', 8: 'no', 9: 'yes', 10: 'no', 11: 'no', 12: 'no', 13: 'yes', 14: 'yes', 15: 'yes', 16: 'yes', 17: 'yes', 18: 'no', 19: 'yes', 20: 'no', 21: 'no', 22: 'yes', 23: 'no', 24: 'no', 25: 'yes', 26: 'yes', 27: 'no', 28: 'no', 29: 'no', 30: 'yes', 31: 'no', 32: 'yes', 33: 'no', 34: 'no', 35: 'no', 36: 'yes', 37: 'no', 38: 'yes', 39: 'no', 40: 'no', 41: 'yes', 42: 'no', 43: 'yes', 44: 'yes', 45: 'yes', 46: 'yes', 47: 'no', 48: 'yes', 49: 'yes', 50: 'yes', 51: 'yes', 52: 'yes', 53: 'yes', 54: 'no', 55: 'yes', 56: 'no', 57: 'yes', 58: 'no', 59: 'no', 60: 'no', 61: 'no', 62: 'no', 63: 'yes', 64: 'no', 65: 'yes', 66: 'yes', 67: 'yes', 68: 'no', 69: 'no', 70: 'yes', 71: 'no', 72: 'no', 73: 'yes', 74: 'yes', 75: 'yes', 76: 'no', 77: 'no', 78: 'yes', 79: 'yes', 80: 'yes', 81: 'no', 82: 'yes', 83: 'yes', 84: 'yes', 85: 'yes', 86: 'no', 87: 'yes', 88: 'no', 89: 'no', 90: 'no', 91: 'no', 92: 'yes', 93: 'yes', 94: 'no', 95: 'yes', 9